The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [77]:
# Read in the input file to a list of strings
with open("input.txt") as f:
    input = f.readlines()

In [71]:
input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""

input = input.split("\n")

In [78]:
# Create a list of all non-period, non-digit symbols in data
symbols = []
for line in input:
    for character in line:
        if character not in symbols and character != "." and not character.isdigit() and character != "\n":
            symbols.append(character)

print(symbols)

['*', '@', '-', '+', '#', '%', '=', '/', '$', '&']


In [79]:
# Create a constant that holds the length of a line
LINE_LENGTH = len(input[0])
print(LINE_LENGTH)

141


In [66]:
class Num_In_Line:
    """
    start_pos is inclusive, end_pos is exclusive
    """
    def __init__(self, num, line, start_pos, end_pos):
        self.num = num
        self.line = line
        self.start_pos = start_pos
        self.end_pos = end_pos

    def __str__(self):
        return f"Num_In_Line({self.num}, {self.line}, {self.start_pos}, {self.end_pos})"

In [88]:
class Symbol_In_Line:
    def __init__(self, symbol, line, pos):
        self.symbol = symbol
        self.line = line
        self.pos = pos

    def __str__(self):
        return f"Symbol_In_Line({self.symbol}, {self.line}, {self.pos})"

In [89]:
# Create a list of all the numbers and symbols in the input
numbers = []
line_symbols = []

for line in input:
    for i in range(len(line)):
        if line[i].isdigit():
            start_pos = i
            for j in range(i, len(line)):
                if not line[j].isdigit():
                    end_pos = j
                    break

            # Only add a number if it's starting position is not within another number in the same line
            if not any([(num.start_pos <= start_pos < num.end_pos) and  num.line == input.index(line) for num in numbers]):
                numbers.append(Num_In_Line(int(line[start_pos:end_pos]), input.index(line), start_pos, end_pos))
        
        elif line[i] in symbols:
            line_symbols.append(Symbol_In_Line(line[i], input.index(line), i))

In [94]:
to_add: list[Num_In_Line] = []

def add_item(item: Num_In_Line):
    # Only allow an addition if the item's line, start_pos, and end_pos combination is unique
    for i in to_add:
        if (item.line == i.line) and (item.start_pos == i.start_pos) and (item.end_pos == i.end_pos):
            return False
    to_add.append(item)

# For each Num_In_Line, check if it is somehow adjacent to a Symbol_In_Line (including diagonally)
# If it is, add its value to to_add
for num in numbers:
    for symbol in line_symbols:
        # Check above
        if num.line - 1 == symbol.line:
            # Check diagonally left
            if num.end_pos == symbol.pos:
                add_item(num)
            # Check diagonally right
            elif num.start_pos == symbol.pos + 1:
                add_item(num)
            # Check directly above
            elif num.start_pos <= symbol.pos < num.end_pos:
                add_item(num)

        # Check below
        elif num.line + 1 == symbol.line:
            # Check diagonally left
            if num.end_pos == symbol.pos:
                add_item(num)
            # Check diagonally right
            elif num.start_pos == symbol.pos + 1:
                add_item(num)
            # Check directly below
            elif num.start_pos <= symbol.pos < num.end_pos:
                add_item(num)

        # Check same line
        elif num.line == symbol.line:
            # Check directly left
            if num.end_pos == symbol.pos:
                add_item(num)
            # Check directly right
            elif num.start_pos == symbol.pos + 1:
                add_item(num)


In [95]:
# Print the sum of all the numbers in to_add
print(sum([to_add[i].num for i in range(len(to_add))]))

531561


The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [97]:
# For each symbol in line_symbols, if it is a '*', add it to part_2_symbols
part_2_symbols = {}
for symbol in line_symbols:
    if symbol.symbol == "*":
        part_2_symbols[symbol] = []

def add_to_star(item: Num_In_Line, key: Symbol_In_Line):
    # Only allow an addition if the item's line, start_pos, and end_pos combination is unique
    for i in part_2_symbols[key]:
        if (item.line == i.line) and (item.start_pos == i.start_pos) and (item.end_pos == i.end_pos):
            return False
    part_2_symbols[key].append(item)

# For each Num_In_Line, check if it is somehow adjacent to a Symbol_In_Line in part_2_symbols.keys (including diagonally)
# If it is, add its value to that key's value list
for num in numbers:
    for symbol in part_2_symbols.keys():
        # Check above
        if num.line - 1 == symbol.line:
            # Check diagonally left
            if num.end_pos == symbol.pos:
                add_to_star(num, symbol)
            # Check diagonally right
            elif num.start_pos == symbol.pos + 1:
                add_to_star(num, symbol)
            # Check directly above
            elif num.start_pos <= symbol.pos < num.end_pos:
                add_to_star(num, symbol)

        # Check below
        elif num.line + 1 == symbol.line:
            # Check diagonally left
            if num.end_pos == symbol.pos:
                add_to_star(num, symbol)
            # Check diagonally right
            elif num.start_pos == symbol.pos + 1:
                add_to_star(num, symbol)
            # Check directly below
            elif num.start_pos <= symbol.pos < num.end_pos:
                add_to_star(num, symbol)

        # Check same line
        elif num.line == symbol.line:
            # Check directly left
            if num.end_pos == symbol.pos:
                add_to_star(num, symbol)
            # Check directly right
            elif num.start_pos == symbol.pos + 1:
                add_to_star(num, symbol)

In [98]:
to_add_star = []

# For each key in part_2_symbols, if its value list has exactly 2 items, add the product of those items to to_add_star.
for key in part_2_symbols.keys():
    if len(part_2_symbols[key]) == 2:
        to_add_star.append(part_2_symbols[key][0].num * part_2_symbols[key][1].num)

In [ ]:
# Print the sum of all the numbers in to_add_star
print(sum(to_add_star))